# FEMA Hazus Flood Damage Functions

The Hazus Flood Model methodology for estimating direct physical damage (e.g., repair costs) to the general building stock is fairly simple and straightforward. For a given census block, each occupancy class (and foundation type) has an appropriate damage function assigned to it (i.e., 1-story, no basement), and computed water depths are used to determine the associated percent damage. This percent damage is multiplied by the full (and depreciated) replacement value of the occupancy class in question to produce an estimate of total full (and depreciated) dollar loss. The “damage states” are derived from the percent damage (e.g., 1-10% damage is considered slight, 11-50% damage is considered moderate, and 51-100% is considered substantial damage.

https://www.fema.gov/media-library-data/20130726-1820-25045-8292/hzmh2_1_fl_tm.pdf

In General:
RES1 = Single Family Dwelling
RES2 = Mobile Home
RES3 = Multi Family Dwelling
with subclass letters for number of units classes

COM1 = Retail Trade
COM2 = Wholesale Trade
COM3 = Personal and Repair
COM4 = Business/Professional/Technical Services
COM5 = Depository Institutions
COM6 = Hospital
COM7 = Medical Office/Clinic
COM8 = Entertainment & Recreation
COM9 = Theaters

For New Orleans sub dataset

Duration definitions:
Hurricane flooding, long duration (one week), salt water
Hurricane flooding, short duration (one day), salt water
Riverine or rainfall flooding, long duration (two or three days), freshwater
Riverine or rainfall flooding, short duration (one day or less), freshwater

In [1]:
import pandas as pd

hazus_flood_file = 'flood_depth.tsv'

df = pd.read_csv(hazus_flood_file,sep='\t')

In [2]:
df.describe()

,DmgFnId,damage,depth
count,35511.000000,35511.000000,35511.000000
mean,305.675256,59.567824,9.904030
std,179.859105,38.742767,8.323025
min,1.000000,0.000000,-4.000000
25%,149.000000,21.000000,3.000000
50%,304.000000,70.000000,10.000000
75%,450.000000,100.000000,17.000000
max,657.000000,100.000000,24.000000


In [3]:
df['Damage'] = pd.to_numeric(df['damage'])
df['Depth'] = pd.to_numeric(df['depth'])
#df

In [4]:
brief = df.filter(items=['Occupancy', 'Source', 'Description', 'Occupy_Class', 'Damage', 'Depth'])
brief

,Occupancy,Source,Description,Occupy_Class,Damage,Depth
1,RES1,USACE - Wilmington,"two story, Pile foundation, structure",RES,3,-4.0
2,RES1,USACE - Wilmington,"two story, Structure",RES,0,-4.0
3,RES1,FIA,"one floor, no basement, Structure, A-Zone",RES,0,-4.0
4,RES1,FIA (MOD.),"one floor, w/ basement, Structure, A-Zone",RES,7,-4.0
5,RES1,FIA,"two floors, no basement, Structure, V-Zone",RES,0,-4.0
6,RES1,USACE - Wilmington,"two story w/ 1/2 living area below, Structure",RES,0,-4.0
7,RES1,FIA,"one floor, no basement, Structure, V-Zone",RES,0,-4.0
8,RES1,FIA (MOD.),"one floor, w/ basement, Structure, V-Zone",RES,7,-4.0
9,RES1,FIA,"split level, no basement, Structure, V-Zone",RES,0,-4.0
10,RES1,FIA (MOD.),"two floors, w/ basement, Structure, V-Zone",RES,4,-4.0


## Extract by Type
fia_res = df[(df.Occupancy == 'GOV1')  & (df.Cover_Class == 'Bldg')].filter(items=['Occupancy', 'DmgFnId', 'Description', 'Occupy_Class', 'damage', 'depth'])
fia_res.sort('Description')

In [5]:
fl_res = df[(df.Occupy_Class == 'RES') & (df.Source == 'USACE - New Orleans') & (df.Cover_Class == 'Bldg')].filter(items=['Occupancy', 'DmgFnId', 'Description', 'Occupy_Class', 'damage', 'depth'])

Apparently, there are multiple damage functions with a single ID, varying by threat, as well as by depth/damage

But even extracting by all known variable within New Orleans Res, we are getting duplicates with multiple values for the same depth.

In [6]:
multifamily = df[(df.Occupancy == 'RES3') & (df.Cover_Class == 'Bldg')].filter(items=['Occupancy', 'DmgFnId', 'Description', 'Occupy_Class', 'Damage', 'Depth'])

In [7]:
multifamily.sort('Depth').sort('Description')

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,Occupancy,DmgFnId,Description,Occupy_Class,Damage,Depth
18893,RES3,204,"Apartment Unit Grade, Structure",Other_Occupy,46,10.0
10073,RES3,204,"Apartment Unit Grade, Structure",Other_Occupy,28,3.0
6293,RES3,204,"Apartment Unit Grade, Structure",Other_Occupy,15,0.0
13853,RES3,204,"Apartment Unit Grade, Structure",Other_Occupy,40,6.0
31493,RES3,204,"Apartment Unit Grade, Structure",Other_Occupy,56,20.0
7553,RES3,204,"Apartment Unit Grade, Structure",Other_Occupy,16,1.0
30233,RES3,204,"Apartment Unit Grade, Structure",Other_Occupy,55,19.0
22673,RES3,204,"Apartment Unit Grade, Structure",Other_Occupy,49,13.0
12593,RES3,204,"Apartment Unit Grade, Structure",Other_Occupy,31,5.0
8813,RES3,204,"Apartment Unit Grade, Structure",Other_Occupy,25,2.0


In [8]:
fl_res.sort_values('depth').sort_values('DmgFnId')

,Occupancy,DmgFnId,Description,Occupy_Class,damage,depth
25299,RES1,141,"one story, Pier foundation, structure, fresh w...",RES,80,16.0
3879,RES1,141,"one story, Pier foundation, structure, fresh w...",RES,1,-1.0
16479,RES1,141,"one story, Pier foundation, structure, fresh w...",RES,78,9.0
11439,RES1,141,"one story, Pier foundation, structure, fresh w...",RES,73,5.0
8919,RES1,141,"one story, Pier foundation, structure, fresh w...",RES,53,3.0
15219,RES1,141,"one story, Pier foundation, structure, fresh w...",RES,73,8.0
26559,RES1,141,"one story, Pier foundation, structure, fresh w...",RES,80,17.0
1359,RES1,141,"one story, Pier foundation, structure, fresh w...",RES,0,-3.0
13959,RES1,141,"one story, Pier foundation, structure, fresh w...",RES,73,7.0
31599,RES1,141,"one story, Pier foundation, structure, fresh w...",RES,80,21.0


In [9]:
fl_res.Occupancy.unique()

array(['RES1', 'RES2'], dtype=object)

In [10]:
fl_res.Description.value_counts()

Mobile Home, structure, salt water, long duration            87
Mobile Home, structure, fresh water, short duration          87
one story, Slab foundation, structure, salt water, long      58
one story, Slab foundation, structure, fresh water, short    58
Mobile Home, structure, salt water, short duration           58
two story, Pier foundation, structure, fresh water, short    58
two story, Slab foundation, structure, salt water, long      58
one story, Pier foundation, structure, fresh water, short    58
Mobile Home, structure, fresh water, long duration           58
one story, Pier foundation, structure, salt water, long      58
two story, Slab foundation, structure, fresh water, long     58
one story, Pier foundation, structure, salt water, short     29
two story, Pier foundation, structure, salt  water, long     29
one story, Pier foundation, structure, fresh water, long     29
two story, Slab foundation, structure, fresh water, short    29
one story, Slab foundation, structure, f

Problem noted: some entries have four items, with no foundation type info

While others have five entries, including this

So...need to separate these line types to know how exactly to parse into columns

In [11]:
fl_res

,Occupancy,DmgFnId,Description,Occupy_Class,damage,depth
60,RES1,142,"one story, Pier foundation, structure, fresh w...",RES,0,-4.0
61,RES1,143,"one story, Slab foundation, structure, fresh w...",RES,0,-4.0
62,RES1,144,"one story, Slab foundation, structure, fresh w...",RES,0,-4.0
63,RES1,145,"one story, Pier foundation, structure, salt wa...",RES,0,-4.0
68,RES1,150,"one story, Structure, fresh water, long duration",RES,0,-4.0
69,RES1,151,"one story, Structure, salt water, short duration",RES,0,-4.0
70,RES1,152,"one story, Structure, salt water, long duration",RES,0,-4.0
71,RES1,153,"one story, Pier foundation, structure, fresh w...",RES,0,-4.0
72,RES1,154,"one story, Slab foundation, structure, fresh w...",RES,0,-4.0
73,RES1,155,"one story, Pier foundation, structure, salt wa...",RES,0,-4.0


In [12]:
for index, r in fl_res.iterrows():
    s = r['Description'].title().split(',')
    structure = '' #nothing useful, so dump into a placeholder
    desc_type = len(s)
    if desc_type == 4:
        r['Stories'], structure, r['Threat'], r['Duration'] = s
        r['Foundation'] = 'Other Foundation'
    if desc_type == 5:
        r['Stories'], r['Foundation'], structure, r['Threat'], r['Duration'] = s
    fl_res.loc[index, 'Foundation'] = r['Foundation']

    # r is only a view of the fl_res, so need to update the real row using the index
    fl_res.loc[index, 'Stories'] = r['Stories'].strip()
    fl_res.loc[index, 'Threat'] = r['Threat'].strip()
    # clean up multiple names for duration here
    #print(r['duration'])
    if 'Long' in r['Duration']  :
        r['Duration'] = 'Long Duration'
    if 'Short' in r['Duration']:
        r['Duration'] = 'Short Duration'
    #print(r['Foundation'])
    s = str(r['Foundation']).strip()
    #print('Length of ', s, ' is ', len(s))
    if len(s) == 3 :
        s = 'Other Foundation'
        #print('Changed to ', r['Foundation'])
    fl_res.loc[index, 'Foundation'] = s

    fl_res.loc[index, 'Duration'] = r['Duration']


In [13]:
fl_res

,Occupancy,DmgFnId,Description,Occupy_Class,damage,depth,Foundation,Stories,Threat,Duration
60,RES1,142,"one story, Pier foundation, structure, fresh w...",RES,0,-4.0,Pier Foundation,One Story,Fresh Water,Long Duration
61,RES1,143,"one story, Slab foundation, structure, fresh w...",RES,0,-4.0,Slab Foundation,One Story,Fresh Water,Short Duration
62,RES1,144,"one story, Slab foundation, structure, fresh w...",RES,0,-4.0,Slab Foundation,One Story,Fresh Water,Long Duration
63,RES1,145,"one story, Pier foundation, structure, salt wa...",RES,0,-4.0,Pier Foundation,One Story,Salt Water,Short Duration
68,RES1,150,"one story, Structure, fresh water, long duration",RES,0,-4.0,Other Foundation,One Story,Fresh Water,Long Duration
69,RES1,151,"one story, Structure, salt water, short duration",RES,0,-4.0,Other Foundation,One Story,Salt Water,Short Duration
70,RES1,152,"one story, Structure, salt water, long duration",RES,0,-4.0,Other Foundation,One Story,Salt Water,Long Duration
71,RES1,153,"one story, Pier foundation, structure, fresh w...",RES,0,-4.0,Pier Foundation,One Story,Fresh Water,Short Duration
72,RES1,154,"one story, Slab foundation, structure, fresh w...",RES,0,-4.0,Slab Foundation,One Story,Fresh Water,Short Duration
73,RES1,155,"one story, Pier foundation, structure, salt wa...",RES,0,-4.0,Pier Foundation,One Story,Salt Water,Long Duration


In [14]:
# create a unique key for each record based on hash of stories, foundation, threat, and duration
for index, r in fl_res.iterrows():
    hashStr = str(r['DmgFnId']) + '-'
    hashStr += str(r['Stories']) + '-'
    hashStr += str(r['Foundation']) + '-'
    hashStr += str(r['Threat']) + '-'
    hashStr += str(r['Duration'])    
    r['hash'] =  hashStr

    fl_res.loc[index, 'Class'] = r['hash']
    #fl_res['class'] = category(r['hash'])
    #.astype('category')
#convert at class level    
fl_res['Class'] = fl_res['Class'].astype('category') 

In [15]:
fl_res['Class']

60       142-One Story-Pier Foundation-Fresh Water-Long...
61       143-One Story-Slab Foundation-Fresh Water-Shor...
62       144-One Story-Slab Foundation-Fresh Water-Long...
63       145-One Story-Pier Foundation-Salt Water-Short...
68       150-One Story-Other Foundation-Fresh Water-Lon...
69       151-One Story-Other Foundation-Salt Water-Shor...
70       152-One Story-Other Foundation-Salt Water-Long...
71       153-One Story-Pier Foundation-Fresh Water-Shor...
72       154-One Story-Slab Foundation-Fresh Water-Shor...
73       155-One Story-Pier Foundation-Salt Water-Long ...
74       156-One Story-Slab Foundation-Salt Water-Long ...
75       157-Two Story-Pier Foundation-Fresh Water-Shor...
76       158-Two Story-Pier Foundation-Fresh Water-Long...
99       141-One Story-Pier Foundation-Fresh Water-Shor...
100      168-Two Story-Other Foundation-Salt Water-Long...
101      169-Two Story-Pier Foundation-Fresh Water-Shor...
102      170-Two Story-Slab Foundation-Fresh Water-Long.

In [31]:
# Generate RESIDENTIAL plots, csvs and cga file

import matplotlib.pyplot as plt
import numpy as np
# for linear interpolation
from scipy.interpolate import interp1d


# try looping through groups then
fl_res.sort_values(by='DmgFnId')
groupByClass = fl_res.groupby('Class')

# get the maximum function id as a tie-breaker
#print(fl_res.groupby(['Class'], sort=False)['DmgFnId'].max())


"""
for name, group in groupByClass:
    bld_type_df = groupByClass.get_group(name)
    filename = 'Residential-' + name.replace(' ', '-')

    header = ['DmgFnId','depth','damage']

    bld_type_df.to_csv(filename + '.csv', columns = header, index=False)
    X = []
    y = []
    color = []
    for index, r in bld_type_df.iterrows():
        fig, ax = plt.subplots( nrows=1, ncols=1 )  # create figure & 1 axis
        X.append(r.depth)
        y.append(r.damage)
        color.append(r.DmgFnId)
        ax.plot(X, y, label=color)
        ax.set_title(name.title())
        ax.set_xlabel('Depth in Feet') 
        ax.set_ylabel('Percent Damage') 
        #ax.legend()
        fig.savefig(filename + '.png')
        plt.close(fig)
"""
for name, group in groupByClass:
    bld_type_df = groupByClass.get_group(name)
    fig, ax = plt.subplots( nrows=1, ncols=1 )  # create figure & 1 axis

    X = []
    y = []
    hazusCurveNumber = ''    
    for index, r in bld_type_df.iterrows():
        X.append(r.depth)
        y.append(r.damage)
        hazusCurveNumber = r.DmgFnId #a bit wasteful...only need one
    ax.plot(X, y)

    # always do simple linear interpolation
    f = interp1d(X, y, fill_value='extrapolate')
    y_lin = f(depthSamples)
    
    cgaColumns(y_lin)
    ax.plot(depthSamples, y_lin)


    cubic = False
    if cubic:
        # use cubic spline fit to model damage curves over standard intervale xb to xe (feet)

        # fit a curve (default order k=3, with smoothing s)
        # returns B-spline
        # as a tuple (t,c,k) containing the vector of knots, the B-spline coefficients, and the degree of the spline
        tck = interpolate.splrep(X, y, s=1, xb=-5.0, xe =30)
        #tck
        ynew = interpolate.splev(xnew, tck, der=0)
        
        ax.plot(xnew, ynew)
        #ax.legend(['Linear', 'Cubic Spline'])


    #ax.plot.show()
    # make a pretty name from the file name
    ax.set_title(filename.replace('-',' ').title())
    ax.set_xlabel('Depth in Feet') 
    ax.set_ylabel('Percent Damage') 
    fig.savefig(filename + '.png')
    plt.close(fig)
        
    #filename = name.replace(' ', '-')
    filename = 'Residential-' + name.replace(' ', '-')

    header = ['DmgFnId','depth','damage']

    bld_type_df.to_csv(filename + '.csv', columns = header, index=False)

#now write a single summary CGA string file
writeCGAfile('res')



## Same for Commercial

In [17]:
fl_com = df[(df.Occupy_Class == 'COM') & (df.Source == 'USACE - New Orleans') & (df.Cover_Class == 'Bldg')].filter(items=['Occupancy', 'DmgFnId', 'Description', 'Occupy_Class', 'damage', 'depth'])

In [18]:
fl_com.sort_values('depth').sort_values('DmgFnId')

,Occupancy,DmgFnId,Description,Occupy_Class,damage,depth
35591,COM1,276,"Bakery, structure, salt water, long duration",COM,100,24.0
7871,COM1,276,"Bakery, structure, salt water, long duration",COM,100,2.0
19211,COM1,276,"Bakery, structure, salt water, long duration",COM,100,11.0
21731,COM1,276,"Bakery, structure, salt water, long duration",COM,100,13.0
22991,COM1,276,"Bakery, structure, salt water, long duration",COM,100,14.0
11651,COM1,276,"Bakery, structure, salt water, long duration",COM,100,5.0
24251,COM1,276,"Bakery, structure, salt water, long duration",COM,100,15.0
34331,COM1,276,"Bakery, structure, salt water, long duration",COM,100,23.0
33071,COM1,276,"Bakery, structure, salt water, long duration",COM,100,22.0
5351,COM1,276,"Bakery, structure, salt water, long duration",COM,0,0.0


In [19]:
fl_com['Landuse'] = ''  #create empty column to start
fl_com['Threat'] = ''  #create empty column to start
fl_com['Duration'] = ''  #create empty column to start
fl_com['Class'] = ''  #create empty column to start
for index, r in fl_com.iterrows():
    s = r['Description'].title().split(',')
    structure = '' #nothing useful, so dump into a placeholder
    desc_type = len(s)
    if desc_type == 4:
        r['Landuse'], structure, r['Threat'], r['Duration'] = s
        #r['Foundation'] = 'Other Foundation'
    else:
        #print('Ooops - what now? String is: ', s)
        r['Landuse'], r['Threat'], r['Duration'] = s


    # r is only a view of the fl_com, so need to update the real row using the index
    fl_com.loc[index, 'Landuse'] = r['Landuse'].strip()
    fl_com.loc[index, 'Threat'] = r['Threat'].strip()
    # clean up multiple names for duration here
    #print(r['duration'])
    if 'Long' in r['Duration']  :
        r['Duration'] = 'Long Duration'
    if 'Short' in r['Duration']:
        r['Duration'] = 'Short Duration'

    fl_com.loc[index, 'Duration'] = r['Duration']


In [20]:
fl_com

,Occupancy,DmgFnId,Description,Occupy_Class,damage,depth,Landuse,Threat,Duration,Class
221,COM1,307,"Gas Station, structure, salt water, short dura...",COM,0,-4.0,Gas Station,Salt Water,Short Duration,
222,COM1,308,"Gas Station, structure, fresh water, long dura...",COM,0,-4.0,Gas Station,Fresh Water,Long Duration,
223,COM1,309,"Gas Station, structure, fresh water, short dur...",COM,0,-4.0,Gas Station,Fresh Water,Short Duration,
224,COM1,310,"Gas Station, structure, composite water & dura...",COM,0,-4.0,Gas Station,Structure,Composite Water & Duration,
225,COM1,311,"Large Grocery, structure, salt water, long dur...",COM,0,-4.0,Large Grocery,Salt Water,Long Duration,
226,COM1,312,"Large Grocery, structure, salt water, short du...",COM,0,-4.0,Large Grocery,Salt Water,Short Duration,
227,COM1,313,"Large Grocery, structure, fresh water, long du...",COM,0,-4.0,Large Grocery,Fresh Water,Long Duration,
228,COM1,314,"Large Grocery, structure, fresh water, short ...",COM,0,-4.0,Large Grocery,Fresh Water,Short Duration,
229,COM1,315,"Large Grocery, structure, composite water & du...",COM,0,-4.0,Large Grocery,Structure,Composite Water & Duration,
230,COM1,316,"Neigborhood Grocery, structure, salt water, lo...",COM,0,-4.0,Neigborhood Grocery,Salt Water,Long Duration,


In [21]:
# create a unique key for each record based on hash
for index, r in fl_com.iterrows():
    r['hash'] = str((r['Landuse'] ) + '-' + str(r['Threat']) + '-' + str(r['Duration']))
    fl_com.loc[index, 'Class'] = r['hash']
#convert at class level    
fl_com['Class'] = fl_com['Class'].astype('category') 

# DOR Use Codes

try to join up values with fl_dor_landuse_codes.csv

result = pd.merge(left, right, on='key')

In [22]:
dor_lut = 'fl_dor_landuse_codes.csv'

dor_df = pd.read_csv(dor_lut,sep=',')
dor_df.drop('Unnamed: 3', axis=1).drop('Unnamed: 4', axis=1)

,DOR_Class,DOR_Code,DOR_Desc
0,Residential,0,Vacant Residential
1,Residential,1,Single Family
2,Residential,2,Mobile Homes
3,Residential,3,Multi-family - 10 units or more
4,Residential,4,Condominiums
5,Residential,5,Cooperatives
6,Residential,6,Retirement Homes not eligible for exemption.
7,Residential,7,"Miscellaneous Residential (migrant camps, boar..."
8,Residential,8,Multi-family - less than 10 units
9,Residential,9,Residential Common Elements/Areas


In [23]:
from difflib import SequenceMatcher as SM
import difflib as DL

In [24]:
# need a loose match..
# but misses 'gas stations' = 'Service stations'
fl_com['DOR_Desc'] = 'Stores, one story' # defailt for unknown commercial
#fl_com['Score'] = '0.0'
dor_desc_list = dor_df['DOR_Desc'].tolist()
#print(dor_desc_list)
for index, r in fl_com.iterrows():
    # look for each word of landuse in DOR lut
    hazus_landuse = r['Landuse']
    best_match = DL.get_close_matches(hazus_landuse, dor_desc_list, 1)
    #print(best_match)
    if best_match:
        fl_com.loc[index, 'DOR_Desc'] = best_match[0]
    else:
        if hazus_landuse == 'Gas Station':
            fl_com.loc[index, 'DOR_Desc'] = 'Service Stations'
        if hazus_landuse == 'Large Grocery':
            fl_com.loc[index, 'DOR_Desc'] = 'Supermarkets'
        if hazus_landuse == "Accounting Firm" or hazus_landuse == 'Legal Office':
            fl_com.loc[index, 'DOR_Desc'] = 'Professional service buildings'
        if hazus_landuse == "Dentist'S Office" or hazus_landuse == 'Medical Office':
            fl_com.loc[index, 'DOR_Desc'] = 'Professional service buildings'
        if hazus_landuse == "Real Estate Office":
            fl_com.loc[index, 'DOR_Desc'] = 'Professional service buildings'
        if hazus_landuse == "Full-Service Restaurant":
            fl_com.loc[index, 'DOR_Desc'] = 'Restaurants, cafeterias'
        if hazus_landuse == "Fast Food Restaurant":
            fl_com.loc[index, 'DOR_Desc'] = 'Drive-in Restaurants'
        if hazus_landuse == "Bank":
            fl_com.loc[index, 'DOR_Desc'] = 'Financial institutions (banks, saving and loan...'
        # Hospitals
            
    print(hazus_landuse, ' ~= ', fl_com.loc[index, 'DOR_Desc'])      

Gas Station  ~=  Service Stations
Gas Station  ~=  Service Stations
Gas Station  ~=  Service Stations
Gas Station  ~=  Service Stations
Large Grocery  ~=  Supermarkets
Large Grocery  ~=  Supermarkets
Large Grocery  ~=  Supermarkets
Large Grocery  ~=  Supermarkets
Large Grocery  ~=  Supermarkets
Neigborhood Grocery  ~=  Stores, one story
Neigborhood Grocery  ~=  Stores, one story
Neigborhood Grocery  ~=  Stores, one story
Neigborhood Grocery  ~=  Stores, one story
Neigborhood Grocery  ~=  Stores, one story
Home Repair Store  ~=  Stores, one story
Home Repair Store  ~=  Stores, one story
Home Repair Store  ~=  Stores, one story
Home Repair Store  ~=  Stores, one story
Home Repair Store  ~=  Stores, one story
Liquor Store  ~=  Stores, one story
Liquor Store  ~=  Stores, one story
Liquor Store  ~=  Stores, one story
Liquor Store  ~=  Stores, one story
Liquor Store  ~=  Stores, one story
Shoe Store  ~=  Stores, one story
Shoe Store  ~=  Stores, one story
Shoe Store  ~=  Stores, one story
Sh

In [25]:
fl_com_dor = pd.merge(fl_com, dor_df, on='DOR_Desc')
fl_com_dor

,Occupancy,DmgFnId,Description,Occupy_Class,damage,depth,Landuse,Threat,Duration,Class,DOR_Desc,DOR_Class,DOR_Code,Unnamed: 3,Unnamed: 4
0,COM1,311,"Large Grocery, structure, salt water, long dur...",COM,0,-4.0,Large Grocery,Salt Water,Long Duration,Large Grocery-Salt Water-Long Duration,Supermarkets,Commercial,14,NaN,NaN
1,COM1,312,"Large Grocery, structure, salt water, short du...",COM,0,-4.0,Large Grocery,Salt Water,Short Duration,Large Grocery-Salt Water-Short Duration,Supermarkets,Commercial,14,NaN,NaN
2,COM1,313,"Large Grocery, structure, fresh water, long du...",COM,0,-4.0,Large Grocery,Fresh Water,Long Duration,Large Grocery-Fresh Water-Long Duration,Supermarkets,Commercial,14,NaN,NaN
3,COM1,314,"Large Grocery, structure, fresh water, short ...",COM,0,-4.0,Large Grocery,Fresh Water,Short Duration,Large Grocery-Fresh Water-Short Duration,Supermarkets,Commercial,14,NaN,NaN
4,COM1,315,"Large Grocery, structure, composite water & du...",COM,0,-4.0,Large Grocery,Structure,Composite Water & Duration,Large Grocery-Structure- Composite Water & Dur...,Supermarkets,Commercial,14,NaN,NaN
5,COM1,311,"Large Grocery, structure, salt water, long dur...",COM,0,-3.0,Large Grocery,Salt Water,Long Duration,Large Grocery-Salt Water-Long Duration,Supermarkets,Commercial,14,NaN,NaN
6,COM1,312,"Large Grocery, structure, salt water, short du...",COM,0,-3.0,Large Grocery,Salt Water,Short Duration,Large Grocery-Salt Water-Short Duration,Supermarkets,Commercial,14,NaN,NaN
7,COM1,313,"Large Grocery, structure, fresh water, long du...",COM,0,-3.0,Large Grocery,Fresh Water,Long Duration,Large Grocery-Fresh Water-Long Duration,Supermarkets,Commercial,14,NaN,NaN
8,COM1,314,"Large Grocery, structure, fresh water, short ...",COM,0,-3.0,Large Grocery,Fresh Water,Short Duration,Large Grocery-Fresh Water-Short Duration,Supermarkets,Commercial,14,NaN,NaN
9,COM1,315,"Large Grocery, structure, composite water & du...",COM,0,-3.0,Large Grocery,Structure,Composite Water & Duration,Large Grocery-Structure- Composite Water & Dur...,Supermarkets,Commercial,14,NaN,NaN


In [26]:
def cgaHeader(depthSamples):
    CGAstr = '# CGA Strings with Interpolated Hazus Data\n'
    CGAstr += '#\n'
    CGAstr += '# Based on: \n'
    CGAstr += '# https://hub.urbmet.com/user/geodesign/notebooks/fema/FEMA%20Hazus%20Flood%20Damage%20Functions.ipynb#\n'
    CGAstr += '#\n'
    CGAstr += '# Const Declarations\n'
    CGAstr += '# Each line contains floating point values of percent damage expected at a flood depth\n'
    CGAstr += 'const HazusHeader= "Description;HazusCurveNo;DOR_Code;Threat;Duration;Landuse;'
    
    for n in depthSamples:
        # add variable names to CGA header
        CGAstr += 'd{};'.format(n)
    CGAstr += '"\n' # close out line with double quote and line return
    return(CGAstr)

def cgaColumns(y_lin):
    CGAstr = 'const h{}="'.format(hazusCurveNumber) 
    CGAstr += '{};'.format(name) # long descriptive hash name first
    CGAstr += curveMetadata(hazusCurveNumber) 
    for y in y_lin:
        CGAstr += '{};'.format(y)
    CGAstr += '"\n'
    return(CGAstr)

def curveMetadata(index):
    DOR_Code = fl_com_dor.loc[ index, 'DOR_Code']   
    Threat = fl_com_dor.loc[ index, 'Threat']   
    Duration = fl_com_dor.loc[ index, 'Duration']   
    Landuse = fl_com_dor.loc[ index, 'Landuse']   
    s = '{};{};{};{};{};'.format(index,DOR_Code, Threat, Duration, Landuse)
    return s
        
print(curveMetadata(445))

def writeCGAfile(hazusClass):
    #print(CGAstr)
    target = open('_{}_hazus.cga'.format(hazusClass), 'w')
    target.write(CGAstr)
    target.close()

"""
CGA Function example:

getValueByDORUC(LU,column)= 
	case listIndex(DORUC_Codes,LU)!=-1:
		listItem(column,listIndex(DORUC_Codes,LU))
        
want something like;

getDamage(curveNum)=
    case listIndex
        
"""


445;11;Structure; Composite Water & Duration;Funeral Home;


'\nCGA Function example:\n\ngetValueByDORUC(LU,column)= \n\tcase listIndex(DORUC_Codes,LU)!=-1:\n\t\tlistItem(column,listIndex(DORUC_Codes,LU))\n        \nwant something like;\n\ngetDamage(curveNum)=\n    case listIndex\n        \n'

In [27]:
# Generate COMMERCIAL plots, csvs and cga file

#from sklearn import linear_model
import matplotlib.pyplot as plt
import numpy as np
# for cubic interpolation
from scipy import interpolate
# for linear interpolation
from scipy.interpolate import interp1d

# try looping through groups then
fl_com.sort_values(by='DmgFnId')

# group by our hashed unique class values
groupByClass = fl_com.groupby('Class')

# set up range for standard evaluations of the spline
depthSamples = np.arange(-5, 31, 1.0)

CGAstr = cgaHeader(depthSamples)

for name, group in groupByClass:
    bld_type_df = groupByClass.get_group(name)
    fig, ax = plt.subplots( nrows=1, ncols=1 )  # create figure & 1 axis

    X = []
    y = []
    hazusCurveNumber = ''    
    for index, r in bld_type_df.iterrows():
        X.append(r.depth)
        y.append(r.damage)
        hazusCurveNumber = r.DmgFnId #a bit wasteful...only need one
    ax.plot(X, y)

    # always do simple linear interpolation
    f = interp1d(X, y, fill_value='extrapolate')
    y_lin = f(depthSamples)
    
    cgaColumns(y_lin)
    ax.plot(depthSamples, y_lin)


    cubic = False
    if cubic:
        # use cubic spline fit to model damage curves over standard intervale xb to xe (feet)

        # fit a curve (default order k=3, with smoothing s)
        # returns B-spline
        # as a tuple (t,c,k) containing the vector of knots, the B-spline coefficients, and the degree of the spline
        tck = interpolate.splrep(X, y, s=1, xb=-5.0, xe =30)
        #tck
        ynew = interpolate.splev(xnew, tck, der=0)
        
        ax.plot(xnew, ynew)
        #ax.legend(['Linear', 'Cubic Spline'])


    #ax.plot.show()
    # make a pretty name from the file name
    ax.set_title(filename.replace('-',' ').title())
    ax.set_xlabel('Depth in Feet') 
    ax.set_ylabel('Percent Damage') 
    fig.savefig(filename + '.png')
    plt.close(fig)
        
    #filename = name.replace(' ', '-')
    filename = 'Commercial-' + name.replace(' ', '-')

    #header = ['CurveID','Occupancy','Occupy_Class','damage','depth','Foundation','Stories','Threat','Duration']
    header = ['DmgFnId','depth','damage']

    bld_type_df.to_csv(filename + '.csv', columns = header, index=False)

#now write a single summary CGA string file
writeCGAfile('com')


/usr/lib/python3/dist-packages/scipy/interpolate/interpolate.py:552: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]


TypeError: an integer is required (got type str)

## Industrial

In [ ]:
fl_ind = df[(df.Occupy_Class == 'IND') & (df.Source == 'USACE - New Orleans') & (df.Cover_Class == 'Bldg')].filter(items=['Occupancy', 'DmgFnId', 'Description', 'Occupy_Class', 'damage', 'depth'])

In [ ]:
fl_ind

In [ ]:
gen_ind = df[(df.Occupy_Class == 'IND') & (df.Cover_Class == 'Bldg')].filter(items=['Occupancy', 'DmgFnId', 'Description', 'Occupy_Class', 'damage', 'depth'])

In [ ]:
gen_ind

## Create standardized interpolations along depth-damage curves for easy CGA lookup

In [ ]:
from scipy import interpolate

x = [-5, 0, 5, 10, 25, 30]
y = [0,10,20,50,80, 100]

# bottom and top of interval to fit
# in our case, representing flood depth in feet
xb = -5.0
xe = 30

# fit a curve (default order k=3, with smoothing s)
# returns B-spline
# as a tuple (t,c,k) containing the vector of knots, the B-spline coefficients, and the degree of the spline
tck = interpolate.splrep(x, y, s=1, xb=-5.0, xe =30)
tck

In [ ]:
# set up range for standard evaluations of the spline
xnew = np.arange(-5, 31, 1.0)
print(xnew)
ynew = interpolate.splev(xnew, tck, der=0)


In [ ]:
plt.figure()
plt.plot(x, y, 'x', xnew, ynew, xnew, np.sin(xnew), x, y, 'b')
plt.legend(['Linear', 'Cubic Spline', 'True'])
#plt.axis([-0.05, 6.33, -1.05, 1.05])
plt.title('Cubic-spline interpolation')
plt.show()